In [1]:
# Importación de librerias

import gzip
import pandas as pd
import ast
from pandas import json_normalize

### Extracción de datos

In [2]:
# Variable con la ruta del archivo
path = '../Datasets/users_items.json.gz'

# Lista para guardar los datos
data = []

# Lectura del archivo
with gzip.open(path, 'rt', encoding='utf-8') as file:
    # Lectura de cada línea del archivo
    for line in file:
        # Conversión de la línea a un diccionario
        try:
            json_data = ast.literal_eval(line)
            data.append(json_data)
        # Manejo de errores
        except ValueError as e:
            print(f"Error en la línea: {line}")
            continue

df = pd.DataFrame(data)

In [3]:
df

,user_id,items_count,steam_id,user_url,items
0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
1,js41637,888,76561198035864385,http://steamcommunity.com/id/js41637,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
2,evcentric,137,76561198007712555,http://steamcommunity.com/id/evcentric,"[{'item_id': '1200', 'item_name': 'Red Orchest..."
3,Riot-Punch,328,76561197963445855,http://steamcommunity.com/id/Riot-Punch,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
4,doctr,541,76561198002099482,http://steamcommunity.com/id/doctr,"[{'item_id': '300', 'item_name': 'Day of Defea..."
...,...,...,...,...,...
88305,76561198323066619,22,76561198323066619,http://steamcommunity.com/profiles/76561198323...,"[{'item_id': '413850', 'item_name': 'CS:GO Pla..."
88306,76561198326700687,177,76561198326700687,http://steamcommunity.com/profiles/76561198326...,"[{'item_id': '11020', 'item_name': 'TrackMania..."
88307,XxLaughingJackClown77xX,0,76561198328759259,http://steamcommunity.com/id/XxLaughingJackClo...,[]
88308,76561198329548331,7,76561198329548331,http://steamcommunity.com/profiles/76561198329...,"[{'item_id': '304930', 'item_name': 'Unturned'..."


### Eliminación de columnas y normalización

In [4]:
# Columnas innecesarias
df.drop(columns=['steam_id', 'user_url'], inplace=True)

In [5]:
df.head()

,user_id,items_count,items
0,76561197970982479,277,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
1,js41637,888,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
2,evcentric,137,"[{'item_id': '1200', 'item_name': 'Red Orchest..."
3,Riot-Punch,328,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
4,doctr,541,"[{'item_id': '300', 'item_name': 'Day of Defea..."


In [6]:
# Desanidamos los datos de la columna items, obteniendo una lista de dataframes.
df_list = [json_normalize(user, 'items', ['user_id', 'items_count']) for user in data]

# Concatenamos todos los dataframes.
df = pd.concat(df_list, ignore_index=True)

In [7]:
df.head()

,item_id,item_name,playtime_forever,playtime_2weeks,user_id,items_count
0,10,Counter-Strike,6.0,0.0,76561197970982479,277
1,20,Team Fortress Classic,0.0,0.0,76561197970982479,277
2,30,Day of Defeat,7.0,0.0,76561197970982479,277
3,40,Deathmatch Classic,0.0,0.0,76561197970982479,277
4,50,Half-Life: Opposing Force,0.0,0.0,76561197970982479,277


In [8]:
# Columnas innecesarias

df.drop(columns=['playtime_2weeks', 'item_id'], inplace=True)

In [9]:
df.head()

,item_name,playtime_forever,user_id,items_count
0,Counter-Strike,6.0,76561197970982479,277
1,Team Fortress Classic,0.0,76561197970982479,277
2,Day of Defeat,7.0,76561197970982479,277
3,Deathmatch Classic,0.0,76561197970982479,277
4,Half-Life: Opposing Force,0.0,76561197970982479,277


### Transformaciones

In [10]:
df.isna().sum()

item_name           0
playtime_forever    0
user_id             0
items_count         0
dtype: int64

In [11]:
df.duplicated().value_counts()

False    5083231
True       69978
Name: count, dtype: int64

In [12]:
# Eliminación de duplicados
df.drop_duplicates(inplace=True)

In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 5083231 entries, 0 to 5153208
Data columns (total 4 columns):
 #   Column            Dtype  
---  ------            -----  
 0   item_name         object 
 1   playtime_forever  float64
 2   user_id           object 
 3   items_count       object 
dtypes: float64(1), object(3)
memory usage: 193.9+ MB


In [16]:
df.shape

(5083231, 4)

In [17]:
df.head()

,item_name,playtime_forever,user_id,items_count
0,Counter-Strike,6.0,76561197970982479,277
1,Team Fortress Classic,0.0,76561197970982479,277
2,Day of Defeat,7.0,76561197970982479,277
3,Deathmatch Classic,0.0,76561197970982479,277
4,Half-Life: Opposing Force,0.0,76561197970982479,277


In [18]:
# Cambios de tipo para optimizar el uso de memoria.
df['items_count'] = df['items_count'].astype('int16')

df['playtime_forever'] = df['playtime_forever'].astype('int16')

### Nota: dejamos user_id como tipo object ya que no vamos a operar sobre sus valores, solo lo utilizamos para hacer los merge entre dataframes.

In [19]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 5083231 entries, 0 to 5153208
Data columns (total 4 columns):
 #   Column            Dtype 
---  ------            ----- 
 0   item_name         object
 1   playtime_forever  int16 
 2   user_id           object
 3   items_count       int16 
dtypes: int16(2), object(2)
memory usage: 135.7+ MB


In [20]:
df.head()

,item_name,playtime_forever,user_id,items_count
0,Counter-Strike,6,76561197970982479,277
1,Team Fortress Classic,0,76561197970982479,277
2,Day of Defeat,7,76561197970982479,277
3,Deathmatch Classic,0,76561197970982479,277
4,Half-Life: Opposing Force,0,76561197970982479,277


### Exportación del dataset

In [21]:
# Utilizamos parquet y el método de compresión snappy que proporciona una gran velocidad de lectura.
df.to_parquet('../Datasets/users_items_preprocessed.parquet', compression='snappy', index=False)

In [22]:
# df.to_parquet('../Datasets/users_items_preprocessed_uc.parquet', index=False)

In [24]:
# import time

# # Archivo sin comprimir
# start_time = time.time()
# df_uncompressed = pd.read_parquet('../Datasets/users_items_preprocessed_uc.parquet')
# uncompressed_time = time.time() - start_time

# # Archivo comprimido con Snappy
# start_time = time.time()
# df_compressed = pd.read_parquet('../Datasets/users_items_preprocessed.parquet')
# compressed_time = time.time() - start_time

# print(f"Tiempo de lectura sin comprimir: {uncompressed_time} segundos")
# print(f"Tiempo de lectura comprimido con Snappy: {compressed_time} segundos")

Tiempo de lectura sin comprimir: 1.2062773704528809 segundos
Tiempo de lectura comprimido con Snappy: 0.9289548397064209 segundos
